In [ ]:
%run discount_common.ipynb

In [ ]:
output_table = "my_catalog.db.test_table"
output_table_bloom = "my_catalog.db.test_table_bloom"

# Initialize Spark session
spark = Utility.get_spark_session()

In [ ]:
import random
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, StringType

# Initialize lists to store data
data = []
for i in range(1, 1000001):  # product_id from 1 to 100
    price = round(random.uniform(10.0, 100.0), 2)  # random price between 10 and 100
    email = f"user_{round(random.uniform(10.0, 100.0))}@email{i}.com"
    data.append((i, price, email))

random.shuffle(data)

# Define schema
schema = StructType([
    StructField("product_id", IntegerType(), False),
    StructField("price", FloatType(), False),
    StructField("email", StringType(), False)
    
])

# Create DataFrame
df = spark.createDataFrame(data, schema)
df.count()
df = df. repartition(20)
IcebergUtility.delete_table(output_table, spark)
IcebergUtility.write_as_table_with_bloom(df, output_table_bloom, "email", "1000")
IcebergUtility.write_as_table(df, output_table)
IcebergUtility.list_tables("my_catalog","db", spark)
IcebergUtility.describe_table(output_table, spark)
IcebergUtility.describe_table(output_table_bloom, spark)
df.count()

In [ ]:
import time
import random
from pyspark.sql import SparkSession

def run_performance_test(iterations:int, table:str):
    query_times = []
    for i in range(1, iterations):  # product_id from 1 to 100
        # random_number1 = random.randint(1000, 1_000_000)
        random_number2 = random.randint(10, 100)
        random_number3 = random.randint(1, 1_000_000)
        spark = Utility.get_spark_session()
        spark.catalog.clearCache()
        
        sql_query = f"""
            SELECT count(1)
            FROM {table} where email in ('user_{random_number2}@email{random_number3}.com')
            """
        start_time = time.time()
        df = spark.sql(sql_query)
        # Trigger the execution (action)
        df.count()
        end_time = time.time()
        spark.stop()
        query_times.append((end_time - start_time) * 1000)
    
    average_ms = sum(query_times) / len(query_times)
    print(f"Average query execution time for {iterations} iterations on {table} : {average_ms:.2f} ms")


run_performance_test(100,  output_table_bloom)
run_performance_test(100, output_table)
run_performance_test(100,  output_table_bloom)
run_performance_test(100, output_table)
# run_performance_test(1000,  output_table_bloom)
# run_performance_test(1000,  output_table)
# run_performance_test(10000,  output_table_bloom)
# run_performance_test(10000,  output_table)
# Bloom filter gives faster results is not proven yet by this notebook.

In [ ]:
spark.stop

In [ ]:
%%bash
rm -rf warehouse
rm -rf spark-warehouse